# Processing marker-gene data in qiime2, part1

**Environment:** qiime2-2021.4

## How to use this notebook:
1. Activate the `qiime2-2021.4` conda environment.
    ```
   source $HOME/miniconda3/bin/activate # use the path in your local machine to activate miniconda
   conda activate qiime2-2021.4 # activate !qiime2 conda environment
    ```
    
2. Launch Jupyter notebook:
    ```
   jupyter notebook
    ```  

In [ ]:
## Hide excessive warnings (optional):
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## change working directory to the project root directory
%cd ..

##  Import data

###  Run1

In [ ]:
# Importing the demultiplexed fastq files
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path data/raw/fastq/run1 \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path data/intermediate/qiime2/demux_run1.qza

# Summarize sequence data and visulaize reads quality
!qiime demux summarize \
  --i-data data/intermediate/qiime2/demux_run1.qza \
  --o-visualization data/intermediate/qiime2/demux_run1.qzv

###  Run2

In [ ]:
# Importing the demultiplexed fastq files
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path data/raw/fastq/run2 \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path data/intermediate/qiime2/demux_run2.qza

# Summarize sequence data and visulaize reads quality
!qiime demux summarize \
  --i-data data/intermediate/qiime2/demux_run2.qza \
  --o-visualization data/intermediate/qiime2/demux_run2.qzv

## Sequence Denoising  wtih DADA2 

###  Run1

In [ ]:
# sequence denoising
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs data/intermediate/qiime2/demux_run1.qza \
  --p-trim-left-f 17 \
  --p-trim-left-r 21 \
  --p-trunc-len-f 287 \
  --p-trunc-len-r 241 \
  --p-min-overlap 20 \
  --p-pooling-method 'pseudo' \
  --p-chimera-method 'pooled' \
  --p-n-threads 16 \
  --o-table data/intermediate/qiime2/table_run1.qza \
  --o-representative-sequences data/intermediate/qiime2/rep_seqs_run1.qza \
  --o-denoising-stats data/intermediate/qiime2/stats_run1.qza \
  --verbose

# sequence denoising summary
!qiime metadata tabulate \
  --m-input-file data/intermediate/qiime2/stats_run1.qza \
  --o-visualization data/intermediate/qiime2/stats_run1.qzv

# feature table summary
!qiime feature-table summarize \
  --i-table data/intermediate/qiime2/table_run1.qza \
  --m-sample-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/table_run1.qzv 

###  Run2

In [ ]:
# sequence denoising
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs data/intermediate/qiime2/demux_run2.qza \
  --p-trim-left-f 17 \
  --p-trim-left-r 21 \
  --p-trunc-len-f 287 \
  --p-trunc-len-r 241 \
  --p-min-overlap 20 \
  --p-pooling-method 'pseudo' \
  --p-chimera-method 'pooled' \
  --p-n-threads 16 \
  --o-table data/intermediate/qiime2/table_run2.qza \
  --o-representative-sequences data/intermediate/qiime2/rep_seqs_run2.qza \
  --o-denoising-stats data/intermediate/qiime2/stats_run2.qza \
  --verbose

# sequence denoising summary
!qiime metadata tabulate \
  --m-input-file data/intermediate/qiime2/stats_run2.qza \
  --o-visualization data/intermediate/qiime2/stats_run2.qzv

# feature table summary
!qiime feature-table summarize \
  --i-table data/intermediate/qiime2/table_run2.qza \
  --m-sample-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/table_run2.qzv 

## Compare results between sequencing runs

### Compute core metrics results without phylogeny

In [ ]:
# run1
!qiime diversity core-metrics \
  --i-table data/intermediate/qiime2/table_run1.qza \
  --m-metadata-file data/metadata.tsv \
  --p-sampling-depth 64843 \
  --output-dir data/intermediate/qiime2/core_metrics_results_run1

# run2
!qiime diversity core-metrics \
  --i-table data/intermediate/qiime2/table_run2.qza \
  --m-metadata-file data/metadata.tsv \
  --p-sampling-depth 71094 \
  --output-dir data/intermediate/qiime2/core_metrics_results_run2

### Procrustes analysis

In [ ]:
# Procrustes analysis
!qiime diversity procrustes-analysis \
  --i-reference data/intermediate/qiime2/core_metrics_results_run1/bray_curtis_pcoa_results.qza \
  --i-other data/intermediate/qiime2/core_metrics_results_run2/bray_curtis_pcoa_results.qza \
  --output-dir data/intermediate/qiime2/compare_runs 

# Procrustes plot
!qiime emperor procrustes-plot \
  --i-reference-pcoa data/intermediate/qiime2/core_metrics_results_run1/bray_curtis_pcoa_results.qza \
  --i-other-pcoa data/intermediate/qiime2/core_metrics_results_run2/bray_curtis_pcoa_results.qza \
  --i-m2-stats data/intermediate/qiime2/compare_runs/disparity_results.qza \
  --m-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/compare_runs/procrustes_plot.qzv

### Mantel test

In [ ]:
!qiime diversity mantel \
  --i-dm1 data/intermediate/qiime2/core_metrics_results_run1/bray_curtis_distance_matrix.qza \
  --i-dm2 data/intermediate/qiime2/core_metrics_results_run2/bray_curtis_distance_matrix.qza \
  --p-method pearson \
  --p-label1 run1_bray_curtis_distance \
  --p-label2 run2_bray_curtis_distance \
  --o-visualization data/intermediate/qiime2/compare_runs/mantel_test.qzv

## Merge data

In [ ]:
# feature table
!qiime feature-table merge \
  --i-tables data/intermediate/qiime2/table_run1.qza \
  --i-tables data/intermediate/qiime2/table_run2.qza \
  --p-overlap-method sum \
  --o-merged-table data/intermediate/qiime2/table_merged.qza

# representative sequences
!qiime feature-table merge-seqs \
  --i-data data/intermediate/qiime2/rep_seqs_run1.qza \
  --i-data data/intermediate/qiime2/rep_seqs_run2.qza \
  --o-merged-data data/intermediate/qiime2/rep_seqs_merged.qza

## Taxonomic  assignment

### Import reference sequence and taxonomy

In [ ]:
!qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path data/reference/silva_132_99_16S.fna \
  --output-path data/intermediate/qiime2/99_otus_silva132.qza

!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path data/reference/silva_132_consensus_taxonomy_l7.txt \
  --output-path data/intermediate/qiime2/ref_taxonomy_silva132.qza

### Extract V3-4 reference sequences

In [ ]:
%%time
!qiime feature-classifier extract-reads \
  --i-sequences data/intermediate/qiime2/99_otus_silva132.qza \
  --p-f-primer CCTACGGGNGGCWGCAG \
  --p-r-primer GACTACHVGGGTATCTAATCC \
  --p-n-jobs 16 \
  --o-reads data/intermediate/qiime2/ref_seqs_silva132.qza

### Train the feature classifier

In [ ]:
%%time
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads data/intermediate/qiime2/ref_seqs_silva132.qza \
  --i-reference-taxonomy data/intermediate/qiime2/ref_taxonomy_silva132.qza \
  --o-classifier data/intermediate/qiime2/silva132_99otu_v3_v4_classifier.qza

### Assign taxonomy

In [ ]:
%%time
!qiime feature-classifier classify-sklearn \
  --i-classifier data/intermediate/qiime2/silva132_99otu_v3_v4_classifier.qza \
  --i-reads data/intermediate/qiime2/rep_seqs_merged.qza \
  --p-n-jobs 16 \
  --o-classification data/intermediate/qiime2/taxonomy_silva132.qza

### Visualize taxonomy 

In [ ]:
# taxonomy file
!qiime metadata tabulate \
  --m-input-file data/intermediate/qiime2/taxonomy_silva132.qza \
  --o-visualization data/intermediate/qiime2/taxonomy_silva132.qzv

# taxonomic barplot
!qiime taxa barplot \
  --i-table data/intermediate/qiime2/table_merged.qza \
  --i-taxonomy data/intermediate/qiime2/taxonomy_silva132.qza \
  --m-metadata-file data/metadata.tsv \
  --o-visualization data/intermediate/qiime2/taxa_bar_plots.qzv